# ML for PM & Trading
---
## Project Proposal: Study of the Links Between Gold Market and US Debt

### This project aims to explore the relationships between the gold market, US debt, and other macroeconomic data such as Federal Reserve (FED) rates using machine learning techniques. By analyzing historical data and applying various ML and Statistical models, we hope to uncover patterns and insights that can inform portfolio management and trading strategies.

- ``Authors`` : Paul-Antoine FRUCHTENREICH & Bilal BENHANA 
- ``Date`` : 2024

## API and package init

In [ ]:
#pip install kagglehub
#pip install seaborn
#pip install statsmodels

In [ ]:
import requests
import json
import pandas as pd
import kagglehub
import matplotlib.pyplot as plt
import datetime

In [ ]:
# Dictionary to map dataset IDs to their corresponding endpoints

dataset_endpoints = {
    'debt_to_penny': 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v2/accounting/od/debt_to_penny',
    'average_interest_rate': 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v2/accounting/od/avg_interest_rates',
    'tips_cpi': 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/tips_cpi_data_detail'
}

def fetch_data(dataset_id, start_date, end_date, sort='record_date'):
    """
    Fetch data from the specified dataset endpoint with optional parameters.

    :param dataset_id: str, ID of the dataset to fetch (e.g., 'debt_to_penny', 'average_interest_rate', 'tips_cpi')
    :param start_date: str, start date for the data in 'YYYY-MM-DD' format
    :param end_date: str, end date for the data in 'YYYY-MM-DD' format
    :param sort: str, field to sort the data by (default is 'record_date')
    :return: dict, JSON response from the API
    """
    days = (datetime.datetime.strptime(end_date, '%Y-%m-%d') - datetime.datetime.strptime(start_date, '%Y-%m-%d')).days
    
    if days > 10000:
        raise ValueError(f"The maximum range of data that can be fetched is 10000 days (approximately 27 years). Here, the range is {days} days. (approximately {days//365} years).")

    if dataset_id not in dataset_endpoints:
        raise ValueError(f"Invalid dataset_id. Available options are: {list(dataset_endpoints.keys())}")

    url = dataset_endpoints[dataset_id]
    params = {
        'filter': f'{sort}:gte:{start_date},{sort}:lte:{end_date}',
        'sort': sort,
        'page[number]': 1,
        'page[size]': days
    }
    response = requests.get(url, params=params)

    if response.status_code != 200:
        raise Exception(f"Failed to fetch data: {response.status_code} - {response.text}")

    df = pd.DataFrame(json.loads(response.text)['data'])
    return df

# Example usage:

"""

debt_to_penny_df = fetch_data('debt_to_penny', '2020-01-01', '2021-01-01')
average_interest_rate_df = fetch_data('average_interest_rate', '2020-01-01', '2021-01-01')
tips_cpi_df = fetch_data('tips_cpi', '1998-01-01', '2020-01-01', sort='original_issue_date')

"""

## Load gold prices and US debt data

### Load the gold price data into a DataFrame

In [ ]:
# Download kaggle dataset of gold price from 1979 to 2022 on a daily basis
path = kagglehub.dataset_download("jishnukoliyadan/gold-price-1979-present")

# The gold with volume, open, high, low, close price  on a daily basis from 1994 to 2021
path_2 = kagglehub.dataset_download("nward7/gold-historical-datasets")

In [ ]:
gold_data = pd.read_csv(path + "/Daily_Gold_Price_on_World.csv")
gold_data_usd = gold_data[["Date", "US dollar (USD)"]]
gold_data_usd['Date'] = pd.to_datetime(gold_data_usd['Date'])
gold_data_usd = gold_data_usd.rename(columns={"US dollar (USD)": "usd_price"})
gold_data_usd = gold_data_usd.set_index('Date')

In [ ]:
gold_data_2 = pd.read_csv(path_2 + "/Gold_Daily .csv")
gold_data_2['Date'] = pd.to_datetime(gold_data_2['Date'])
gold_data_2 = gold_data_2.rename(columns={"Change %": "Daily_return", "Vol":"Volume"})
gold_data_2 = gold_data_2.set_index('Date')
# convert the volume to numeric
gold_data_2["Volume"] = gold_data_2["Volume"].str.replace("K", "e3")
gold_data_2["Volume"] = pd.to_numeric(gold_data_2["Volume"], errors='coerce')
gold_data_2.dropna()

### Load US debt into a dataset

In [ ]:
# Fetching US debt from 1993 to 2023
debt_to_penny_df_1 = fetch_data('debt_to_penny', '1993-01-01', '2003-01-01')
debt_to_penny_df_2 = fetch_data('debt_to_penny', '2003-01-01', '2023-01-01')
debt_to_penny_df = pd.concat([debt_to_penny_df_1, debt_to_penny_df_2])

# keep only the relevant columns
debt_to_penny_df = debt_to_penny_df[['record_date', 'tot_pub_debt_out_amt']]

# convert  tot_pub_debt_out_amt to numeric in trillions
debt_to_penny_df['record_date'] = pd.to_datetime(debt_to_penny_df['record_date'])
debt_to_penny_df['tot_pub_debt_out_amt'] = pd.to_numeric(debt_to_penny_df['tot_pub_debt_out_amt']) / 1e12  # convert to trillions $
debt_to_penny_df = debt_to_penny_df.set_index('record_date')


#### Periods of interest

In [ ]:
# Define some periods of interest
decenal_periods = {
    '1993-2003': ('1993-01-01', '2003-01-01'),
    '2003-2013': ('2003-01-01', '2013-01-01'),
    '2013-2023': ('2013-01-01', '2023-01-01')
}

# Define all the crisis periods since 1993
crisis_periods = {
    '1997-1998 Asian Financial Crisis': ('1997-07-01', '1998-12-31'),
    '2000-2002 Dot-Com Bubble Burst': ('2000-03-01', '2002-12-31'),
    '2007-2008 Global Financial Crisis': ('2007-07-01', '2008-12-31'),
    '2010-2012 European Debt Crisis': ('2010-01-01', '2012-12-31'),
    '2015-2016 Chinese Stock Market Crash': ('2015-06-01', '2016-12-31'),
    '2019-2020 COVID-19 Pandemic': ('2019-12-01', '2020-12-31')
}

### Features Engineering

In [ ]:
# compute daily returns
gold_data_usd['daily_return'] = gold_data_usd['usd_price'].pct_change()
debt_to_penny_df['daily_return'] = debt_to_penny_df['tot_pub_debt_out_amt'].pct_change()

# compute daily variation
gold_data_usd['daily_variation'] = gold_data_usd['usd_price'].diff()
debt_to_penny_df['daily_variation'] = debt_to_penny_df['tot_pub_debt_out_amt'].diff()

# compute 7d - 28d - 90d - 365 annualized volatility for gold and debt
gold_data_usd['volatility_7d'] = gold_data_usd['daily_return'].rolling(window=7).std() * ((365/7)**0.5)
gold_data_usd['volatility_28d'] = gold_data_usd['daily_return'].rolling(window=28).std() * ((365/28)**0.5)
gold_data_usd['volatility_90d'] = gold_data_usd['daily_return'].rolling(window=90).std() * ((365/90)**0.5)
gold_data_usd['volatility_365d'] = gold_data_usd['daily_return'].rolling(window=365).std()

## Plots

### Gold Price

In [ ]:
gold_data_usd['usd_price'].plot(color='gold')
plt.title('Gold Price')
plt.xlabel('Date')
plt.ylabel('Price in USD')
plt.show()

In [ ]:
# plot the gold_data_2 price with volume 
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Date')
ax1.set_ylabel('Price', color='gold')
ax1.plot(gold_data_2.index, gold_data_2['Price'], color='gold')
ax1.tick_params(axis='y', labelcolor='gold')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Volume', color='blue')  # we already handled the x-label with ax1
ax2.plot(gold_data_2.index, gold_data_2['Volume'], color='blue')
ax2.tick_params(axis='y', labelcolor='blue')

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.title('Gold Price and Volume')
plt.show()


### US Debt

In [ ]:
debt_to_penny_df['tot_pub_debt_out_amt'].plot()

plt.title('US Debt')
plt.xlabel('Date')
plt.ylabel('Debt in Trillions USD')
plt.show()

### US Debt aside Gold Price during some key periods

In [ ]:
def plot_aside(start_date='1979', end_date='2023', crisis_periods=None):
    fig, ax1 = plt.subplots(figsize=(14, 7))

    # Plot gold price data
    ax1.plot(gold_data_usd[start_date:end_date].index, gold_data_usd[start_date:end_date]['usd_price'], color='gold', label='Gold Price')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Gold Price (USD)', color='gold')
    ax1.tick_params(axis='y', labelcolor='gold')

    # Create a second y-axis to plot US debt data
    ax2 = ax1.twinx()
    ax2.plot(debt_to_penny_df[start_date:end_date].index, debt_to_penny_df[start_date:end_date]['tot_pub_debt_out_amt'], color='blue', label='US Debt')
    ax2.set_ylabel('US Debt (Trillions USD)', color='blue')
    ax2.tick_params(axis='y', labelcolor='blue')

    # Add title and legend
    plt.title(f'Gold Price and US Debt during {crisis_periods}')
    fig.tight_layout()
    fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))

    plt.show()


# Example usage
for period_name, (start_date, end_date) in decenal_periods.items():
    plot_aside(start_date=start_date, end_date=end_date, crisis_periods=period_name)

In [ ]:
# Make a function that do the bar plot of gold_data_2['Volume'] and line chart of debt_to_penny_df['tot_pub_debt_out_amt'] aside for a given parameter

def plot_aside_2(start_date='1979', end_date='2023', crisis_periods=None):
    fig, ax1 = plt.subplots(figsize=(14, 7))

    # Plot gold price data
    ax1.bar(gold_data_2[start_date:end_date].index, gold_data_2[start_date:end_date]['Volume'], color='gold', label='Gold Volume')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Gold Volume', color='gold')
    ax1.tick_params(axis='y', labelcolor='gold')

    # Create a second y-axis to plot US debt data
    ax2 = ax1.twinx()
    ax2.plot(debt_to_penny_df[start_date:end_date].index, debt_to_penny_df[start_date:end_date]['tot_pub_debt_out_amt'], color='blue', label='US Debt')
    ax2.set_ylabel('US Debt (Trillions USD)', color='blue')
    ax2.tick_params(axis='y', labelcolor='blue')

    # Add title and legend
    plt.title(f'Gold Volume and US Debt during {crisis_periods}')
    fig.tight_layout()
    fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))

    plt.show()

# Example usage

for period_name, (start_date, end_date) in decenal_periods.items():
    plot_aside_2(start_date=start_date, end_date=end_date, crisis_periods=period_name)

for period_name, (start_date, end_date) in crisis_periods.items():
    plot_aside_2(start_date=start_date, end_date=end_date, crisis_periods=period_name)

### Histogram of retruns for gold and Debt

In [ ]:
# Histogram of daily returns for gold and debt

fig, ax = plt.subplots(2, 2, figsize=(14, 7))

gold_data_usd['daily_return'].plot.hist(bins=20, ax=ax[0, 0], color='gold')
ax[0, 0].set_title('Gold Daily Returns')
ax[0, 0].set_xlabel('Daily Return')
ax[0, 0].set_ylabel('Frequency')

debt_to_penny_df['daily_return'].plot.hist(bins=20, ax=ax[0, 1], color='blue')
ax[0, 1].set_title('US Debt Daily Returns')
ax[0, 1].set_xlabel('Daily Return')
ax[0, 1].set_ylabel('Frequency')

gold_data_usd['daily_variation'].plot.hist(bins=20, ax=ax[1, 0], color='gold')
ax[1, 0].set_title('Gold Daily Variation')
ax[1, 0].set_xlabel('Daily Variation')
ax[1, 0].set_ylabel('Frequency')

debt_to_penny_df['daily_variation'].plot.hist(bins=20, ax=ax[1, 1], color='blue')
ax[1, 1].set_title('US Debt Daily Variation')
ax[1, 1].set_xlabel('Daily Variation')
ax[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()



### Heatmap of US Debt Variation and Gold variation

In [ ]:
import seaborn as sns

# Resample the data to daily frequency and fill missing values
debt_daily = debt_to_penny_df.resample('D').ffill()

# Create a pivot table with years as rows and days of the year as columns
debt_daily['Year'] = debt_daily.index.year
debt_daily['DayOfYear'] = debt_daily.index.dayofyear
# Calculate the daily variation in debt
debt_daily['Debt_Variation'] = debt_daily['tot_pub_debt_out_amt'].diff()

# Create a pivot table with years as rows and days of the year as columns for debt variation
debt_variation_pivot = debt_daily.pivot('Year', 'DayOfYear', 'Debt_Variation')

# Plot the heatmap for debt variation
plt.figure(figsize=(12, 8))
sns.heatmap(debt_variation_pivot, center=0, cmap='coolwarm', cbar_kws={'label': 'Debt Variation in  USD'})
plt.title('Heatmap of US Debt Variation by Days')
plt.xlabel('Day of Year')
plt.ylabel('Year')
plt.show()

# Resample the gold data to daily frequency and fill missing values
gold_daily = gold_data_usd.resample('D').ffill()

# Create a pivot table with years as rows and days of the year as columns
gold_daily['Year'] = gold_daily.index.year
gold_daily['DayOfYear'] = gold_daily.index.dayofyear
# Calculate the daily variation in gold price
gold_daily['Gold_Variation'] = gold_daily['usd_price'].diff()

# Create a pivot table with years as rows and days of the year as columns for gold price variation
gold_variation_pivot = gold_daily.pivot('Year', 'DayOfYear', 'Gold_Variation')

# Plot the heatmap for gold price variation
plt.figure(figsize=(12, 8))
sns.heatmap(gold_variation_pivot, center=0, cmap='coolwarm', cbar_kws={'label': 'Gold Price Variation in USD (Trillions)'})
plt.title('Heatmap of Gold Price Variation by Days')
plt.xlabel('Day of Year')
plt.ylabel('Year')
plt.show()

# Plot the heatmap for gold volume 
gold_daily_2 = gold_data_2.resample('D').ffill()
gold_daily_2['Year'] = gold_daily_2.index.year
gold_daily_2['DayOfYear'] = gold_daily_2.index.dayofyear
gold_daily_2['Gold_Volume'] = gold_data_2['Volume']

gold_volume_pivot = gold_daily_2.pivot('Year', 'DayOfYear', 'Gold_Volume')

plt.figure(figsize=(12, 8))
sns.heatmap(gold_volume_pivot, center=0, cmap='coolwarm', cbar_kws={'label': 'Gold Volume'})
plt.title('Heatmap of Gold Volume by Days')
plt.xlabel('Day of Year')
plt.ylabel('Year')
plt.show()


### Heatmap of US Daily Return and Gold Daily Return

In [ ]:
# Heatmap of Gold daily returns by year
gold_daily['Gold_Return'] = gold_daily['daily_return']
gold_return_pivot = gold_daily.pivot('Year', 'DayOfYear', 'Gold_Return')

plt.figure(figsize=(12, 8))
sns.heatmap(gold_return_pivot, cmap='coolwarm', center=0, cbar_kws={'label': 'Gold Daily Return'})
plt.title('Heatmap of Gold Daily Returns by Days (1979-2022)')
plt.xlabel('Day of Year')
plt.ylabel('Year')
plt.show()

# Heatmap of Debt daily returns by year
debt_daily['Debt_Return'] = debt_daily['daily_return']
debt_return_pivot = debt_daily.pivot('Year', 'DayOfYear', 'Debt_Return')

plt.figure(figsize=(12, 8))
sns.heatmap(debt_return_pivot, cmap='coolwarm', center=0, cbar_kws={'label': 'Debt Daily Return'})
plt.title('Heatmap of US Debt Daily Returns by Days (1993-2023)')
plt.xlabel('Day of Year')
plt.ylabel('Year')
plt.show()

## Statistical Analysis

### Compute correlations between gold prices and US debt

In [ ]:
def correlation_analysis(periods):
    # Create a DataFrame to store the correlation values
    columns = pd.MultiIndex.from_product([["gold_vs_debt"],["correlation","correlation_daily_return"]])
    correlations = pd.DataFrame(data=[], index=periods.keys(), columns=columns)

    # Calculate and print the correlation for each period
    for period, (start_date, end_date) in periods.items():
        gold_period = gold_data_usd[start_date:end_date]
        debt_period = debt_to_penny_df[start_date:end_date]
        
        # Merge the dataframes on the date index
        merged_data = pd.merge(gold_period, debt_period, left_index=True, right_index=True)
        
        # Calculate the correlation
        correlation = merged_data['usd_price'].corr(merged_data['tot_pub_debt_out_amt'])
        correlations.loc[period,("gold_vs_debt","correlation")] = correlation
        
        # Calculate the correlation of daily returns
        correlation_dr = merged_data['daily_return_x'].corr(merged_data['daily_return_y'])
        correlations.loc[period,("gold_vs_debt","correlation_daily_return")] = correlation_dr
        
    return correlations

def find_best_correlated_periods(first_date, last_date):
    # create a dictionary to store the periods
    periods = {}

    # Look for the best correlation within first and last_date
    for i in range(1, 11):
        start_date = pd.to_datetime(first_date)
        end_date = start_date + pd.DateOffset(years=i)
        if end_date > pd.to_datetime(last_date):
            break
        periods[f'Period {i}'] = (start_date, end_date)

    # Calculate the correlation for each period
    correlations = correlation_analysis(periods)
    # Find the period with the highest correlation
    best_period = correlations.idxmax().values[0][0]

    return periods[best_period],correlations




### Merging gold and debt data in one dataframe

In [ ]:
full_data = gold_data_usd.merge(debt_to_penny_df, how="outer", left_index=True, right_index=True)
# rename columns and drop NaN values
full_data = full_data.rename(columns={"tot_pub_debt_out_amt": "debt_amt", "usd_price" : "gold_price", "daily_return_x": "gold_daily_return", "daily_return_y": "debt_daily_return", "daily_variation_x": "gold_daily_variation", "daily_variation_y": "debt_daily_variation"}).dropna()
full_data.head()

### Test de Causalité de Granger

Le test de causalité de Granger est une méthode statistique utilisée pour déterminer si une série temporelle peut prédire une autre série temporelle. Contrairement à une simple analyse de corrélation, qui mesure seulement la force et la direction de la relation linéaire entre deux variables, le test de Granger examine la capacité d'une variable à fournir des informations sur les valeurs futures d'une autre variable.

#### Pertinence par rapport à une analyse de corrélation

1. **Directionnalité** : La corrélation ne donne aucune information sur la direction de la relation. Le test de Granger peut indiquer si X cause Y ou si Y cause X.
2. **Temporalité** : La corrélation est une mesure instantanée et ne prend pas en compte les décalages temporels. Le test de Granger intègre les décalages temporels pour évaluer la relation causale.
3. **Prédiction** : La corrélation ne peut pas être utilisée pour la prédiction, alors que le test de Granger est spécifiquement conçu pour évaluer la capacité prédictive d'une série temporelle sur une autre.

In [ ]:

from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from statsmodels.tsa.api import VAR

# Charger les données
data = full_data[['gold_price', 'debt_amt']]

# Vérifier la stationnarité des séries temporelles
def check_stationarity(series):
    result = adfuller(series)
    print(f'Statistique ADF: {result[0]}')
    print(f'Valeur p: {result[1]}')
    print(f'Valeurs critiques: {result[4]}')
    return result[1] < 0.05  # Retourne True si la série est stationnaire

# Appliquer le test ADF sur les deux séries
is_gold_price_stationary = check_stationarity(data['gold_price'])
is_us_debt_stationary = check_stationarity(data['debt_amt'])

# Différencier les séries si elles ne sont pas stationnaires
if not is_gold_price_stationary:
    data['gold_price'] = data['gold_price'].diff().dropna()
if not is_us_debt_stationary:
    data['debt_amt'] = data['debt_amt'].diff().dropna()

# Supprimer les valeurs manquantes après différenciation
data = data.dropna()

# Visualiser les séries temporelles après nettoyage
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(data['gold_price'], label='Prix de l\'or', color='gold')
plt.legend()
plt.subplot(2, 1, 2)
plt.plot(data['debt_amt'], label='Dette américaine', color='blue')
plt.legend()
plt.show()

check_stationarity(data['gold_price'])
check_stationarity(data['debt_amt'])

# Effectuer le test de causalité de Granger
max_lag = 90  # On teste jusqu'à 90 jour de lags (environ 3 mois)
model = VAR(data[['gold_price', 'debt_amt']])
lag_order = model.select_order(maxlags=max_lag)
print(lag_order.summary())

# Utiliser le nombre optimal de lags pour le test de Granger
optimal_lag = lag_order.aic

result_gold_cause_debt = grangercausalitytests(data[['gold_price', 'debt_amt']], optimal_lag, verbose=False) # Test de causalité de l'or sur la dette
result_debt_cause_gold = grangercausalitytests(data[['debt_amt', 'gold_price']], optimal_lag, verbose=False) # Test de causalité de la dette sur l'or

# Récupérer les p-valeurs des 2 tests de causalité
p_values_gold_cause_debt = [result_gold_cause_debt[i + 1][0]['ssr_ftest'][1] for i in range(optimal_lag)]
p_values_debt_cause_gold = [result_debt_cause_gold[i + 1][0]['ssr_ftest'][1] for i in range(optimal_lag)]

# Visualiser les p-valeurs
plt.figure(figsize=(12, 6))
plt.plot(range(1, optimal_lag + 1), p_values_gold_cause_debt, label='Gold causes Debt', color='green')
plt.plot(range(1, optimal_lag + 1), p_values_debt_cause_gold, label='Debt causes Gold', color='red')
plt.axhline(0.05, color='black', linestyle='--', label='Significance level (0.05)')
plt.xlabel('Lag')
plt.ylabel('P-value')
plt.title('Granger Causality Test Results')
plt.legend()
plt.show()

print(f'le maxlag optimal est pour les données daily est {optimal_lag} jours soit {optimal_lag//30} mois environ')

#### Problème des P-valeurs qui Tendent vers 0

Lorsque les p-valeurs descendent à 0 pour des lags élevés dans un test de causalité de Granger, cela peut être dû à plusieurs facteurs :

1. **Surajustement (Overfitting)** : Avec un nombre élevé de lags, le modèle peut commencer à surajuster les données, capturant non seulement les relations réelles mais aussi le bruit aléatoire. Cela peut entraîner des p-valeurs artificiellement faibles, indiquant une causalité apparente qui n'est pas réellement présente.

2. **Autocorrélation** : Les séries temporelles peuvent avoir une forte autocorrélation, ce qui signifie que les valeurs passées influencent fortement les valeurs futures. Lorsque le nombre de lags est élevé, le modèle peut capturer cette autocorrélation, ce qui peut entraîner des p-valeurs faibles.

3. **Dépendance Spurious** : Avec un grand nombre de lags, il est possible que des relations spurious (fausses) soient détectées. Cela peut se produire lorsque le modèle trouve des corrélations qui ne sont pas réellement causales mais qui apparaissent significatives en raison du grand nombre de paramètres estimés.

4. **Problèmes de Stationnarité** : Si les séries temporelles ne sont pas correctement stationnarisées, cela peut entraîner des résultats trompeurs. Les séries non stationnaires peuvent montrer des tendances ou des cycles qui peuvent être mal interprétés comme des relations causales.

5. **Taille de l'Échantillon** : Avec un grand nombre de lags, le nombre de paramètres à estimer augmente, ce qui peut réduire la puissance statistique du test, surtout si la taille de l'échantillon est limitée. Cela peut également conduire à des p-valeurs faibles de manière artificielle.

### Proposition : Agrégation des Données

Pour atténuer ces problèmes, une solution consiste à agrégater les données à une fréquence plus faible, comme hebdomadaire (weekly) ou mensuelle (monthly). L'agrégation des données peut aider à :

- **Réduire le Bruit** : En lissant les fluctuations quotidiennes, l'agrégation peut réduire le bruit et rendre les tendances sous-jacentes plus visibles.
- **Simplifier le Modèle** : Avec des données agrégées, le nombre de lags nécessaires pour capturer les relations temporelles peut être réduit, diminuant ainsi le risque de surajustement.
- **Améliorer la Stationnarité** : Les données agrégées peuvent être plus stationnaires, facilitant l'interprétation des résultats du test de Granger.


In [ ]:
# Agrégation hebdomadaire
data_weekly = full_data[['gold_price', 'debt_amt']].resample('W').mean()

# Différencier les séries si elles ne sont pas stationnaires
if not check_stationarity(data_weekly['gold_price']):
    data_weekly['gold_price'] = data_weekly['gold_price'].diff().dropna()
if not check_stationarity(data_weekly['debt_amt']):
    data_weekly['debt_amt'] = data_weekly['debt_amt'].diff().dropna()

# Supprimer les valeurs manquantes après différenciation
data_weekly = data_weekly.dropna()

# Utiliser le critère d'information pour déterminer le nombre optimal de lags
max_lag = 90 # On teste jusqu'à 90 semaines de lags (environ 2 ans)
model = VAR(data_weekly[['gold_price', 'debt_amt']])
lag_order = model.select_order(maxlags=max_lag)
optimal_lag = lag_order.aic

# Effectuer le test de causalité de Granger avec le nombre optimal de lags
result_gold_cause_debt = grangercausalitytests(data_weekly[['gold_price', 'debt_amt']], optimal_lag, verbose=False)
result_debt_cause_gold = grangercausalitytests(data_weekly[['debt_amt', 'gold_price']], optimal_lag, verbose=False)

# Récupérer les p-valeurs des 2 tests de causalité
p_values_gold_cause_debt = [result_gold_cause_debt[i + 1][0]['ssr_ftest'][1] for i in range(optimal_lag)]
p_values_debt_cause_gold = [result_debt_cause_gold[i + 1][0]['ssr_ftest'][1] for i in range(optimal_lag)]

# Visualiser les p-valeurs
plt.figure(figsize=(12, 6))
plt.plot(range(1, optimal_lag + 1), p_values_gold_cause_debt, label='Gold causes Debt', color='green')
plt.plot(range(1, optimal_lag + 1), p_values_debt_cause_gold, label='Debt causes Gold', color='red')
plt.axhline(0.05, color='black', linestyle='--', label='Significance level (0.05)')
plt.xlabel('Lag')
plt.ylabel('P-value')
plt.title('Granger Causality Test Results (Weekly Data)')
plt.legend()
plt.show()

print(f'le maxlag optimal est pour les donnée hebdomadaire est {optimal_lag} soit environ {optimal_lag//4} mois')
print(f'la p-value minimale pour la causalité de l\'or sur la dette est {min(p_values_gold_cause_debt)} obtenue pour un lag de {p_values_gold_cause_debt.index(min(p_values_gold_cause_debt))} semaines')

In [ ]:
# Agrégation mensuelle

data_monthly = full_data[['gold_price', 'debt_amt']].resample('M').mean()

# Différencier les séries si elles ne sont pas stationnaires
if not check_stationarity(data_monthly['gold_price']):
    data_monthly['gold_price'] = data_monthly['gold_price'].diff().dropna()
if not check_stationarity(data_monthly['debt_amt']):
    data_monthly['debt_amt'] = data_monthly['debt_amt'].diff().dropna()

# Supprimer les valeurs manquantes après différenciation
data_monthly = data_monthly.dropna()

# Utiliser le critère d'information pour déterminer le nombre optimal de lags

max_lag = 36  # On teste jusqu'à 36 mois de lags (environ 3 ans)
model = VAR(data_monthly[['gold_price', 'debt_amt']])
lag_order = model.select_order(maxlags=max_lag)
optimal_lag = lag_order.aic

# Effectuer le test de causalité de Granger avec le nombre optimal de lags
result_gold_cause_debt = grangercausalitytests(data_monthly[['gold_price', 'debt_amt']], optimal_lag, verbose=False)
result_debt_cause_gold = grangercausalitytests(data_monthly[['debt_amt', 'gold_price']], optimal_lag, verbose=False)

# Récupérer les p-valeurs des 2 tests de causalité
p_values_gold_cause_debt = [result_gold_cause_debt[i + 1][0]['ssr_ftest'][1] for i in range(optimal_lag)]
p_values_debt_cause_gold = [result_debt_cause_gold[i + 1][0]['ssr_ftest'][1] for i in range(optimal_lag)]

# Visualiser les p-valeurs
plt.figure(figsize=(12, 6))
plt.plot(range(1, optimal_lag + 1), p_values_gold_cause_debt, label='Gold causes Debt', color='green')
plt.plot(range(1, optimal_lag + 1), p_values_debt_cause_gold, label='Debt causes Gold', color='red')
plt.axhline(0.05, color='black', linestyle='--', label='Significance level (0.05)')
plt.xlabel('Lag')
plt.ylabel('P-value')
plt.title('Granger Causality Test Results (Monthly Data)')
plt.legend()
plt.show()

print(f'Le maxlag optimal pour les données mensuelles est {optimal_lag} mois')

Quelque soit l'échelle de temps, l'hypothèse G->D semble plus plausible que D->G (cf. interprétation de la p-value)

Disclaimer : cette analyse statistique doit être prise comme consultative uniquement


## ML Model for Signal Detection

Concevoir un modèle ML qui utilise les données du marché de l'or pour obtenir des signaux quant à d'éventuels mouvements non anticipés de la dette américaine pour servir de base à la conception d'un stratégie de trading qui profite de nos prédictions du profil d'endettement US.

---
#### Steps :

- Concevoir un modèle de classification et/ou un modèle de régression.
- Concevoir une stratégie de trading basée sur ces modèles et les données du marché de l'or.
- Bonus : Concevoir une stratégie de trading basée sur les données de la Fed et du profil d'endettement puis comparer les performances avec la stratégie précédente. 

### Model Implementation

- **Features** : `gold_price`, `gold_volume`, `gold_vol7d`, `gold_vol28d`, `gold_vol90d`, `gold_variation`, `gold_return`
- **Target** : Classifier le profil du delta d'endettement à horizon `x` jours/semaines/mois (``remboursement``, ``endettement``, ``surendettement``). Idéalement effectuer une regression pour prédire le delta d'endettement d'ici l'horizon de temps testé.
- **Rational** : Le but est de savoir si des patterns particuliers (e.g. : gros volume échangé, changement soudain de volatilité, ou autre) permettent de prédire des accélérations soudaines ou des remboursements anticipés de la dette américaine et si l'or sert vraiment d'indicateur avancé ou bien si les taux/annonces de la Fed et les valeurs passées du profil d'endettement suffisent pour anticiper ces mouvements.
- **Hyperparemètre** : 
    - `x` (l'horizon de temps sur lequel le signal permet au mieux de prédire un surendettement)
    - `q` quantile qui definit le seuil s à partir des variations relative de la dette horizon `x` jours
    - `s` = `f(q,x)` (le seuil de classification qui permet de discriminer un endettement d'un surendettement) 

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# Chargement des données
gold_data = gold_data_2[['Price', 'Volume', 'Daily_return']].rename(columns={'Price': 'gold_price', 'Volume': 'gold_volume', 'Daily_return': 'gold_return'})
debt_data = debt_to_penny_df[['tot_pub_debt_out_amt']].rename(columns={'tot_pub_debt_out_amt': 'debt_amt'})

# Feature Engineering
gold_data['gold_vol7d'] = gold_data['gold_return'].rolling(window=7).std() * ((365/7)**0.5)
gold_data['gold_vol28d'] = gold_data['gold_return'].rolling(window=28).std() * ((365/28)**0.5)
gold_data['gold_vol90d'] = gold_data['gold_return'].rolling(window=90).std() * ((365/90)**0.5)
gold_data['gold_volume_variation'] = gold_data['gold_volume'].diff().fillna(0)
gold_data['gold_variation'] = gold_data['gold_price'].diff()
gold_data['gold_volume_absolut_variation'] = abs(gold_data['gold_volume_variation'])
gold_data['gold_volume_absolut_variation_7d'] = gold_data['gold_volume_absolut_variation'].rolling(window=7).sum()
gold_data['gold_volume_absolut_variation_28d'] = gold_data['gold_volume_absolut_variation'].rolling(window=28).sum()
gold_data['gold_volume_absolut_variation_90d'] = gold_data['gold_volume_absolut_variation'].rolling(window=90).sum()

# Prétraitement des données
data = pd.merge(gold_data, debt_data, left_index=True, right_index=True)
data.dropna(inplace=True)

# Définition de l'horizon de temps x et du quantile q pour calculer le seuil s
x = 30  # par exemple, 30 jours
q = 0.95  # par exemple, 95%

# Calcul de la variation relative de la dette sur x jours
data['debt_delta'] = (data['debt_amt'].shift(-x) - data['debt_amt']) / data['debt_amt']

# Computation of s which is the top 5% quantile of the debt_delta distribution playing the role of threshold for the classification of the target variable (remboursement, endettement, surendettement)
s = data['debt_delta'].quantile(q)

# histogramme de debt_delta
data['debt_delta'].hist(bins=50)
plt.title(f'Distribution de la variation de la dette horizon {x} jours')
plt.xlabel('Variation de la dette')
plt.ylabel('Fréquence')
plt.show()

# count the number of days where the debt_delta is above the threshold
print(f'On compte {data[data["debt_delta"] > s].shape[0]} jours sur un total de {data.shape[0]} jours où la variation relative de la dette sur les {x} prochains jours est supérieure au seuil {round(s,2)}')

# Classification de la target en 3 classes : remboursement, endettement et surendettement
data['target'] = pd.cut(data['debt_delta'], bins=[-np.inf, 0, s, np.inf], labels=['remboursement', 'endettement', 'surendettement'])

# Suppression des lignes avec des valeurs manquantes dans la target
data.dropna(subset=['target'], inplace=True)

# Séparation des données X contenant les features et y contenant la target
X = data.drop(columns=['debt_delta', 'target', 'debt_amt'])
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Plots gold_data and debt_data aside put vertical red line when target is surendettement

def plot_aside_3(start_date='1979', end_date='2023', crisis_periods=None):
    fig, ax1 = plt.subplots(figsize=(14, 7))

    # Plot gold price data
    ax1.plot(gold_data[start_date:end_date].index, gold_data[start_date:end_date]['gold_price'], color='gold', label='Gold Price')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Gold Price (USD)', color='gold')
    ax1.tick_params(axis='y', labelcolor='gold')

    # Create a second y-axis to plot US debt data
    ax2 = ax1.twinx()
    ax2.plot(debt_data[start_date:end_date].index, debt_data[start_date:end_date]['debt_amt'], color='blue', label='US Debt')
    ax2.set_ylabel('US Debt (Trillions USD)', color='blue')
    ax2.tick_params(axis='y', labelcolor='blue')

    # Add title and legend
    plt.title(f'Gold Price and US Debt during {crisis_periods}')
    fig.tight_layout()
    fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))

    # Add vertical lines for surendettement if the surrounding data is in the period
    surendettement_dates = data[data['target'] == 'surendettement'].index
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    for date in surendettement_dates:
        if start_date < date < end_date:
            ax1.axvline(date, color='red', linestyle='--')
    plt.show()

# Example usage

for period_name, (start_date, end_date) in crisis_periods.items():
    plot_aside_3(start_date=start_date, end_date=end_date, crisis_periods=period_name)

for period_name, (start_date, end_date) in decenal_periods.items():
    plot_aside_3(start_date=start_date, end_date=end_date, crisis_periods=period_name)

In [ ]:

# Entraînement d'un modèle de classification de type Gradient Boosting avec GridSearchCV
def GBoostTraining(data, selected_features,  param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2]
    }):

    # Séparation des données X contenant les features et y contenant la target
    try:
        X = data[selected_features]
        y = data['target']
    except KeyError:
        raise KeyError('Les colonnes sélectionnées ne sont pas présentes dans le jeu de données')
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Initialisation du modèle
    model = GradientBoostingClassifier(random_state=42)

    # Initialisation de GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)

    # Entraînement du modèle
    grid_search.fit(X_train, y_train)

    # Prédiction sur les données de test
    y_pred = grid_search.predict(X_test)

    # Affichage du rapport de classification
    print(classification_report(y_test, y_pred))
    
    # Affichage des meilleurs hyperparamètres
    print(f'Les meilleurs hyperparamètres sont: {grid_search.best_params_}')


### Model Optimization

- **Selection des variables** : Ridge Regression
- **Optimisation des hyperparamètres et leur impact sur la feature selection** : Une fois qu'on sait entraîner puis tester les performances d'un modèle, il nous reste à chercher les paramètres `x` et `q` optimaux (Cross-Validation) et observer si/comment la feature selection établie par le ridge change en fonction de ``x`` et ``q``
- **Redefinition de la target** : la target de classification etait définie par la variation relative de la dette horizon `x` jours mais on pourrait tout à fait imaginer de la definir par rapport à un depassement d'un niveau de volatilité seuil `s` ou variation cumulée pour prendre en compte tous le chemin entre ``t`` et ``t+x``  

- **Bonus** : 
    - **Transformation des données et feature augmentation** : rajouter des features notamment liées à `debt_amt`. Aussi, on peut chercher à transformer nos données (``gold price`` ``inflation-ajusted``, ``debt_amt`` déflaté du ``GDP`` ou GDP/hab)
    - **Incorporation des données macroéconomiques** : Ajouter des indicateurs macroéconomiques disponible grâce à l'API tels que le ``taux de chômage``, ``inflation`` (TIPS and CPI), et les ``taux d'intérêt`` pour améliorer les prédictions.
    - **Analyse comparative des modèles** : De la classification à la regression (logistique). Tester différents types de boosted trees, puis voire si l'implémentation de RNN comme LSTM pour prendre en compte les relations de dépendence temporelle des time series permet d'ameliorer les performances.
    - **Analyse de la saisonnalité** : Étudier les effets saisonniers sur les prix de l'or et la dette américaine -> faire varier l'échelle des données (jusque-là on travaillait avec du daily mais comme on l'a vu dans la partie Granger Causality, il peut être intéressant de revenir à du ``weekly`` ou du ``monthly``) 



#### Variable selection with Ridge Regression

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV

# Séparation des données
X = data.drop(columns=['debt_delta', 'target', 'debt_amt'])
y = data['debt_delta']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardiser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Définir les valeurs possibles pour alpha (échelle logarithmique)
alphas = np.logspace(-1, 5, 30)

# Initialiser le modèle Ridge
ridge = Ridge()

# Utiliser GridSearchCV pour trouver le meilleur alpha
param_grid = {'alpha': alphas}
grid_search = GridSearchCV(ridge, param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

# Meilleur alpha trouvé
best_alpha = grid_search.best_params_['alpha']
print(f"Meilleur alpha trouvé: {best_alpha}")

# Entraîner le modèle Ridge avec le meilleur alpha
ridge_best = Ridge(alpha=best_alpha)
ridge_best.fit(X_train_scaled, y_train)

# Sélectionner les caractéristiques basées sur le modèle Ridge
selector = SelectFromModel(ridge_best, prefit=True)
X_train_selected = selector.transform(X_train_scaled)

# Plot de l'évolution de la significativité des régressions
coefs = []
for alpha in alphas:
    ridge.set_params(alpha=alpha)
    ridge.fit(X_train_scaled, y_train)
    coefs.append(ridge.coef_)

plt.figure(figsize=(10, 6))
plt.plot(np.log10(alphas), coefs)
plt.xlabel('log10(alpha)')
plt.ylabel('Coefficients')
plt.title('Evolution des coefficients en fonction de log(alpha)')
plt.axis('tight')
plt.legend(X.columns)
plt.show()

In [ ]:
# Afficher les caractéristiques sélectionnées
selected_features = X.columns[selector.get_support()]
print("Caractéristiques sélectionnées:", selected_features)

# Prédiction sur les données de test
X_test_selected = selector.transform(X_test_scaled)
y_pred = ridge_best.predict(X_test_scaled)

# Calcul de l'erreur quadratique moyenne
mse = np.mean((y_pred - y_test) ** 2)
print(f"Erreur quadratique moyenne: {mse}")

# Evalualuation du modèle Ridge
plt.figure(figsize=(10, 6))
plt.scatter(y_test.index, y_test, label='True')
plt.scatter(y_test.index, y_pred, label='Predicted')
plt.xlabel('Date')
plt.ylabel('Debt Delta')
plt.title('Prédiction du modèle Ridge')
plt.legend()
plt.show()


#### Training of Boosted Tree with the selected features only

In [ ]:
# Définition des features et de la target
X = data[selected_features]
y = data['target']

# Entraînement d'un modèle de classification de type Gradient Boosting avec GridSearchCV
# Séparation des données

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Définition des hyperparamètres à tester
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Initialisation du modèle
model = GradientBoostingClassifier(random_state=42)

# Initialisation de GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)

# Entraînement du modèle
grid_search.fit(X_train, y_train)

# Prédiction sur les données de test
y_pred = grid_search.predict(X_test)

# Affichage du rapport de classification
print(classification_report(y_test, y_pred))

# Affichage des meilleurs hyperparamètres
print(f'Les meilleurs hyperparamètres sont: {grid_search.best_params_}')


#### Grid-Search like function to Fine Tune the model with ``x`` and ``q``

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error

# Grid-Search like function to Fine Tune the model with x and q
param_grid = {
    'x': [1, 7, 28, 90],
    'q': [0.9, 0.95, .97, 0.99]
}

def fine_tune_model(param_grid, verbose=False):
    best_score = 0
    best_params = None
    best_features = None
    for x in param_grid['x']:
        for q in param_grid['q']:
            # Calcul de la variation relative de la dette sur x jours
            data['debt_delta'] = (data['debt_amt'].shift(-x) - data['debt_amt']) / data['debt_amt']

            # Calcul du seuil s
            s = data['debt_delta'].quantile(q)

            # Classification de la target en 3 classes
            data['target'] = pd.cut(data['debt_delta'], bins=[-np.inf, 0, s, np.inf], labels=['remboursement', 'endettement', 'surendettement'])

            # Suppression des lignes avec des valeurs manquantes dans la target
            data.dropna(subset=['target'], inplace=True)

            # Séparation des données X contenant les features et y contenant la target
            X = data.drop(columns=['debt_delta', 'target', 'debt_amt'])
            y = data['debt_delta']
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            # Regression Ridge pour selectionner les features

            # Standardiser les données
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            # Initialiser le modèle Ridge
            ridge = Ridge()

            # Utiliser GridSearchCV pour trouver le meilleur alpha
            param_grid_ridge = {'alpha': alphas}
            grid_search_ridge = GridSearchCV(ridge, param_grid_ridge, cv=5)
            grid_search_ridge.fit(X_train_scaled, y_train)

            # Meilleur alpha trouvé
            best_alpha = grid_search_ridge.best_params_['alpha']

            # Entraîner le modèle Ridge avec le meilleur alpha
            ridge_best = Ridge(alpha=best_alpha)
            ridge_best.fit(X_train_scaled, y_train)

            # Sélectionner les caractéristiques basées sur le modèle Ridge
            selector = SelectFromModel(ridge_best, prefit=True)
            X_train_selected = selector.transform(X_train_scaled)

            # Plot de l'évolution de la significativité des régressions
            if verbose:
                coefs = []
                for alpha in alphas:
                    ridge.set_params(alpha=alpha)
                    ridge.fit(X_train_scaled, y_train)
                    coefs.append(ridge.coef_)

                plt.figure(figsize=(10, 6))
                plt.plot(np.log10(alphas), coefs)
                plt.xlabel('log10(alpha)')
                plt.ylabel('Coefficients')
                plt.title(f'Evolution des coefficients en fonction de log(alpha) pour x={x} et q={q}')
                plt.axis('tight')
                plt.legend(X.columns)
                plt.show()
            
            # Afficher les caractéristiques sélectionnées
            selected_features = X.columns[selector.get_support()]
            if verbose:
                print(f'Caractéristiques sélectionnées pour x={x} et q={q}: {selected_features}')

            # Evaluation du modèle Ridge
            X_test_selected = selector.transform(X_test_scaled)
            y_pred = ridge_best.predict(X_test_scaled)

            # Compute the score model (here R2 score but can be changed to another metric such as mean_absolute_percentage_error)
            score = r2_score(y_test, y_pred)
            
            if verbose:
                print(f'Score for x={x} and q={q}: {score}')

            if score > best_score:
                best_score = score
                best_params = {'x': x, 'q': q}
                best_features = selected_features
    
    return best_params, best_score, best_features

# Fine tune the model

best_params, best_score, best_features = fine_tune_model(param_grid, verbose=True)
print(f"Meilleurs paramètres trouvés: {best_params}")
print(f"Meilleure score: {best_score}")
print(f"Meilleures caractéristiques sélectionnées: {best_features}")

In [ ]:

from sklearn.linear_model import LinearRegression

# Train a simple regression model with the best parameters

x = best_params['x']
q = best_params['q']

# Prétraitement des données
data = pd.merge(gold_data, debt_data, left_index=True, right_index=True)
data.dropna(inplace=True)

# Calcul de la variation relative de la dette sur x jours
data['debt_delta'] = (data['debt_amt'].shift(-x) - data['debt_amt']) / data['debt_amt']

# Suppression des lignes avec des valeurs manquantes dans debt_delta
data.dropna(subset=['debt_delta'], inplace=True)

# model de regression linéaire avec les features selectionnées
X = data[best_features]
y = data['debt_delta']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# entrainement du modèle de regression linéaire

model = LinearRegression()
model.fit(X_train, y_train)

# Prédiction sur les données de test
y_pred = model.predict(X_test)

# Calcul de l'erreur quadratique moyenne

print(f"Erreur quadratique moyenne: {mean_squared_error(y_test, y_pred)}")
print(f"Score R2: {r2_score(y_test, y_pred)}")
print(f"Mean Absolute Percentage Error: {mean_absolute_percentage_error(y_test, y_pred)}")

# Evaluation du modèle de regression linéaire
plt.figure(figsize=(10, 6))
plt.scatter(y_test.index, y_test, label='True')
plt.scatter(y_test.index, y_pred, label='Predicted')
plt.xlabel('Date')
plt.ylabel('Debt Delta')
plt.title(f'Prédiction du modèle de regression linéaire pour x={x} et q={q}')
plt.legend()
plt.show()

In [ ]:
GBoostTraining(data, best_features)

## Trading Strategy
---
On a un modèle qui nous permet de detecter des signaux concernant les variations futures (horizon `x` jours) de la dette américaine, il ne nous reste plus qu'à trouver une manière d'en tirer profit avec un stratégie de trading. Deux questions : Quoi trader ? Comment charger les positions ?

- **Proxy de la dette américaine** : verfier si on peut encoder la pente de la dette US par le ``spread d'un swap taux US 10y - 1y equi-std`` comme instrument pour notre stratégie de trading
- **Backtesting des stratégies de trading** : Mettre en place un environnement de backtesting pour évaluer les performances des stratégies de trading basées sur les modèles prédictifs et sur un proxy de la dette américaine comme instrument tradable